## 1. Load data

In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/real-time-advertisers-auction/Dataset.csv


In [18]:
df = pd.read_csv(str(dirname)+'/'+str(filename))

In [19]:
df.head(2)

,date,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,order_id,line_item_type_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,total_revenue,viewable_impressions,measurable_impressions,revenue_share_percent
0,2019-06-30 00:00:00,351,10,187,2,84,3473,19,60,1,4,5174,16,0.0,2,16,1
1,2019-06-30 00:00:00,351,10,187,2,84,3473,19,58,1,4,5174,6,0.0,0,6,1


In [20]:
df.isna().sum()

date                       0
site_id                    0
ad_type_id                 0
geo_id                     0
device_category_id         0
advertiser_id              0
order_id                   0
line_item_type_id          0
os_id                      0
integration_type_id        0
monetization_channel_id    0
ad_unit_id                 0
total_impressions          0
total_revenue              0
viewable_impressions       0
measurable_impressions     0
revenue_share_percent      0
dtype: int64

Drop duplicates

In [21]:
df.drop_duplicates().reset_index(drop=True, inplace=True)

make CPM

In [22]:
def weird_division(n, d):
    return n / d if d else 0

In [23]:
foo = lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 

In [24]:
df['CPM'] = df.apply(foo, axis=1)

take CPM >=0 and drop total_revenue, measurable_impressions, revenue_share_percent

In [25]:
df = df.loc[df['CPM']>=0]

In [26]:
df.drop(['total_revenue','revenue_share_percent', 'measurable_impressions'], axis=1, inplace=True)

In [27]:
df.head(2)

,date,site_id,ad_type_id,geo_id,device_category_id,advertiser_id,order_id,line_item_type_id,os_id,integration_type_id,monetization_channel_id,ad_unit_id,total_impressions,viewable_impressions,CPM
0,2019-06-30 00:00:00,351,10,187,2,84,3473,19,60,1,4,5174,16,2,0.0
1,2019-06-30 00:00:00,351,10,187,2,84,3473,19,58,1,4,5174,6,0,0.0


In [28]:
df['CPM'].describe()

count    567290.000000
mean        115.950473
std         741.231817
min           0.000000
25%           0.000000
50%           0.000000
75%         101.818182
max      283620.000000
Name: CPM, dtype: float64

In [29]:
df = df[df['CPM'] < df['CPM'].quantile(.95)] 

## 2.Train test split

In [30]:
split_data = '2019-06-22'

test = df[df.date >= split_data]
train = df[df.date < split_data]

In [36]:
X = train.drop(['date','CPM'], axis=1)
y = train['CPM']

In [37]:
TEST_SIZE = 0.3
RAND_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state=RAND_STATE)

## 3. Model

### 3.1 Default models

In [38]:
models = [Ridge, Lasso, RandomForestRegressor, GradientBoostingRegressor]
model_names = ['Ridge', 'Lasso', 'RandomForestRegressor', 'GradientBoostingRegressor']

def estimate_model(model_cls):
    model = model_cls(random_state=RAND_STATE)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = mean_squared_error(y_test, y_pred)
    return score

regression_scores = []
for model_cls in models:
    score = estimate_model(model_cls)
    regression_scores.append(score)

index = np.array(regression_scores).argmin()
answer = regression_scores[index]
print("best score: %s" % model_names[index])

best score: RandomForestRegressor


### 3.2 Random Forest

In [39]:
rfg = RandomForestRegressor()
rfg.fit(X_train, y_train)

y_pred_rfg = rfg.predict(X_test)

In [40]:
print("MSE: ", (mean_squared_error(y_pred_rfg, y_test)))

SyntaxError: unexpected EOF while parsing (<ipython-input-40-168713486cd9>, line 1)